In [83]:
pip install scikit-surprise


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [84]:
from collections import defaultdict

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import surprise as sp
from sklearn.model_selection import train_test_split

In [85]:
# importar data
ratings_set = pd.read_csv('./dataset/rating.csv')
display(ratings_set)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [86]:
ratings_set.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [87]:
ratings_set.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [88]:
movies_set = pd.read_csv('./dataset/movie.csv')
display(movies_set)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [89]:
movies_set.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [90]:
movies_set.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [91]:
# Combina los DataFrames de calificaciones y detalles de películas en uno solo por 'movieId' de forma interna y muestra el resultado

ratings_movies_set = ratings_set.merge(movies_set, how='inner', on='movieId')
display(ratings_movies_set)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [97]:
# Luego, carga este subconjunto en un objeto `dataset` utilizando la biblioteca Surprise para su análisis posterior.

subset_ratings = ratings_set.sample(frac=1, random_state=42)  

train, test = train_test_split(subset_ratings, test_size=0.2, random_state=42)

reader = sp.Reader(rating_scale=(0, 5))
dataset = sp.Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)


In [98]:
train

,userId,movieId,rating,timestamp
86035,559,329,3.0,845475849
19095,122,106782,5.0,1461561445
53209,352,924,4.0,1493931975
63374,414,3269,3.0,961514310
30415,212,81834,4.0,1490203271
...,...,...,...,...
99583,610,784,5.0,1479544723
91864,596,3000,5.0,1535708657
59035,385,500,3.0,834694199
25661,177,70159,2.5,1435838020


In [99]:
test

,userId,movieId,rating,timestamp
39920,274,4683,4.0,1172275751
31097,217,1259,3.0,955941925
27388,186,1376,5.0,1031079628
35836,243,356,4.0,837155178
12082,74,3273,2.5,1207499293
...,...,...,...,...
70695,452,1265,5.0,1013398330
69352,448,4477,1.0,1028111080
35238,237,7371,3.5,1410632372
46881,307,924,3.5,1186084299


In [100]:
# Definir la cuadrícula de hiperparámetros para GridSearchCV
knn_params_grid = {
    'min_k': [2],
    'k': [10, 30, 40, 50],
    'verbose': [False],
    'sim_options': {
        'name': ['pearson', 'cosine'],
        'min_support': [10, 15, 20],
        'user_based': [True]
    }
}

# Realizar la búsqueda de cuadrícula utilizando GridSearchCV
gs = sp.model_selection.GridSearchCV(
    algo_class=sp.KNNBasic, 
    param_grid=knn_params_grid,
    measures=['rmse'],
    cv=5,  # utilizando 5-fold cross-validation
    n_jobs=-1
)

gs.fit(dataset)
print("Mejor rmse score:", gs.best_score['rmse']) # Imprimir la mejor puntuación RMSE
print("Mejor parametros: ", gs.best_params['rmse']) # Imprimir los mejores parámetros

# Asignar el algoritmo KNN con la mejor precisión para su uso posterior
knn = gs.best_estimator['rmse']

# Ejecutar validación cruzada para evaluar el modelo
cv_results = sp.model_selection.cross_validate(knn, dataset, measures=['rmse'], cv=5, verbose=False, n_jobs=-1)

# Imprimir resultados de validación cruzada
print("RMSE promedio en validación cruzada:", cv_results['test_rmse'].mean())

Mejor rmse score: 0.9822528371900043
Mejor parametros:  {'min_k': 2, 'k': 30, 'verbose': False, 'sim_options': {'name': 'cosine', 'min_support': 10, 'user_based': True}}
RMSE promedio en validación cruzada: 0.9815371519009946


In [101]:
# Entrenar el modelo K-Nearest Neighbors (KNN) utilizando el conjunto completo de entrenamiento.
ds=dataset.build_full_trainset()
knn.fit(ds)

In [81]:
ds.to_inner_iid('87346')

ValueError: Item 87346 is not part of the trainset.

In [102]:
test

,userId,movieId,rating,timestamp
39920,274,4683,4.0,1172275751
31097,217,1259,3.0,955941925
27388,186,1376,5.0,1031079628
35836,243,356,4.0,837155178
12082,74,3273,2.5,1207499293
...,...,...,...,...
70695,452,1265,5.0,1013398330
69352,448,4477,1.0,1028111080
35238,237,7371,3.5,1410632372
46881,307,924,3.5,1186084299


In [103]:
predictions=knn.predict(uid=11, iid=293)
predictions

Prediction(uid=11, iid=293, r_ui=None, est=4.033405926280354, details={'actual_k': 30, 'was_impossible': False})

In [104]:
def predict_ratings(algo, user_id, movie_id):
    # Verificar si el usuario ya calificó la película
    check = ratings_movies_set.query(f"userId == {user_id} and movieId == {movie_id}")
    if not check.empty:
        print(f"Usuario {user_id} ya calificó {movie_id}")
        display(check)  # Mostrar detalles de la calificación existente
    else:
        # Mostrar calificaciones del usuario y la película
        display(f"Calificaciones de usuario {user_id}:", ratings_movies_set[ratings_movies_set['userId'] == user_id])
        display(f"Calificaciones de película {movie_id}:", ratings_movies_set[ratings_movies_set['movieId'] == movie_id])
        
        # Estimar calificación utilizando el modelo
        predict = algo.predict(uid=user_id, iid=movie_id)
        print(f"Estimación para usuario {user_id} y película {movie_id}: ", predict.est)  # Mostrar estimación
        print("Detalles:", predict)  # Mostrar detalles

In [53]:
import matplotlib.pyplot as plt

calificacion_real = predict_ratings(knn, predict, 239).  # Calificación real

# Generar una lista de calificaciones predichas
calificaciones_predichas = []

for test_user_id in test['userId'].unique():
    pred = predict_ratings(knn, test_user_id, movie_id)
    calificaciones_predichas.append(pred)

# Crear un gráfico de dispersión
plt.figure(figsize=(8, 6))
plt.scatter(calificaciones_reales, calificaciones_predichas, alpha=0.5)
plt.title('Predicción Real vs. Predicción')
plt.xlabel('Calificaciones Reales')
plt.ylabel('Calificaciones Predichas')
plt.grid(True)

# Agregar una línea de referencia diagonal para comparación perfecta
plt.plot([min(calificaciones_reales), max(calificaciones_reales)], [min(calificaciones_reales), max(calificaciones_reales)], linestyle='--', color='red')

# Mostrar el gráfico
plt.show()

SyntaxError: invalid syntax (4227750097.py, line 3)

In [105]:
# Realizar una predicción de calificación para el usuario 11 y la película 293 utilizando el modelo KNN.
predict_ratings(knn, 11, 293)


'Calificaciones de usuario 11:'

,userId,movieId,rating,timestamp,title,genres
269,11,6,5.0,902154266,Heat (1995),Action|Crime|Thriller
858,11,110,5.0,902154266,Braveheart (1995),Action|Drama|War
2319,11,349,5.0,902154342,Clear and Present Danger (1994),Action|Crime|Drama|Thriller
2431,11,356,5.0,901200263,Forrest Gump (1994),Comedy|Drama|Romance|War
3003,11,457,5.0,902154316,"Fugitive, The (1993)",Thriller
...,...,...,...,...,...,...
44047,11,1840,4.0,901200111,He Got Game (1998),Drama
44054,11,1882,2.0,901200143,Godzilla (1998),Action|Sci-Fi|Thriller
44087,11,1918,4.0,901200070,Lethal Weapon 4 (1998),Action|Comedy|Crime|Thriller
44120,11,2002,2.0,902154383,Lethal Weapon 3 (1992),Action|Comedy|Crime|Drama


'Calificaciones de película 293:'

,userId,movieId,rating,timestamp,title,genres
30483,6,293,3.0,845553660,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30484,15,293,3.0,1510571962,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30485,16,293,4.0,1377477870,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30486,17,293,3.5,1307262308,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30487,18,293,4.5,1455050049,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
...,...,...,...,...,...,...
30611,603,293,2.0,954482255,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30612,604,293,3.0,832080193,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30613,606,293,4.5,1171387841,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller
30614,608,293,4.0,1117490629,Léon: The Professional (a.k.a. The Professiona...,Action|Crime|Drama|Thriller


Estimación para usuario 11 y película 293:  4.033405926280354
Detalles: user: 11         item: 293        r_ui = None   est = 4.03   {'actual_k': 30, 'was_impossible': False}


In [106]:
# Realizar una predicción de calificación para el usuario 100 y la película 72998 utilizando el modelo KNN.
predict_ratings(knn, 100, 72998)


'Calificaciones de usuario 100:'

,userId,movieId,rating,timestamp,title,genres
227,100,3,3.5,1100183804,Grumpier Old Men (1995),Comedy|Romance
1277,100,223,3.5,1100185924,Clerks (1994),Comedy
1510,100,235,1.0,1100183797,Ed Wood (1994),Comedy|Drama
1868,100,296,3.5,1100184820,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2482,100,356,4.0,1100184829,Forrest Gump (1994),Comedy|Drama|Romance|War
...,...,...,...,...,...,...
87809,100,2262,4.0,1100186135,About Last Night... (1986),Comedy|Drama|Romance
87818,100,3244,4.0,1100184414,"Goodbye Girl, The (1977)",Comedy|Romance
87830,100,5380,4.5,1100184334,"Importance of Being Earnest, The (2002)",Comedy|Drama|Romance
87840,100,6183,4.5,1100184242,Pillow Talk (1959),Comedy|Musical|Romance


'Calificaciones de película 72998:'

,userId,movieId,rating,timestamp,title,genres
42588,10,72998,2.5,1455356351,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42589,15,72998,3.0,1510572052,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42590,18,72998,4.0,1455749238,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42591,21,72998,4.0,1418847046,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42592,22,72998,3.5,1268726172,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
...,...,...,...,...,...,...
42680,599,72998,2.5,1498518529,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42681,601,72998,4.0,1513712155,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42682,605,72998,3.5,1277097079,Avatar (2009),Action|Adventure|Sci-Fi|IMAX
42683,606,72998,3.0,1368460014,Avatar (2009),Action|Adventure|Sci-Fi|IMAX


Estimación para usuario 100 y película 72998:  3.884116538807556
Detalles: user: 100        item: 72998      r_ui = None   est = 3.88   {'actual_k': 30, 'was_impossible': False}


In [107]:
def get_top_n(predictions, n=10):
    # Mapear las predicciones a cada usuario.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Luego, ordenar las predicciones para cada usuario y obtener las n más altas.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [108]:
# Crear un conjunto de entrenamiento completo a partir del conjunto de datos
train_set = dataset.build_full_trainset()

# Crear un conjunto de prueba con todas las combinaciones que no están en el conjunto de entrenamiento
test_set = train_set.build_anti_testset()

# Entrenar el modelo KNN con el conjunto de entrenamiento
knn.fit(train_set)

# Realizar predicciones en el conjunto de prueba
predictions = knn.test(test_set)

# Obtener las mejores 10 recomendaciones para cada usuario
top_10 = get_top_n(predictions, n=10)


In [109]:
# Tomar 50 usuarios aleatorios junto con sus 10 mejores recomendaciones
sample_top_10 = random.sample(top_10.items(), 50)

# Inicializar un contador
counter = 0

# Iterar a través de los usuarios seleccionados aleatoriamente y sus recomendaciones
for uid, user_ratings in sample_top_10:
    counter += 1
    print(f"{counter}. Usuario {uid} - Top 10 recomendaciones:")
    for (iid, rating) in user_ratings:
        # Obtener el título de la película a partir del identificador de la película (iid)
        movie_title = movies_set[movies_set['movieId'] == iid].title.item()
        print(f"{movie_title} (id:{iid}), calificación: {rating}")
    print("\n")


1. Usuario 389 - Top 10 recomendaciones:
Lone Star (1996) (id:800), calificación: 5
Yojimbo (1961) (id:3030), calificación: 4.794700123253426
State and Main (2000) (id:4029), calificación: 4.753981270875222
You Can Count on Me (2000) (id:3983), calificación: 4.753981270875222
21 Jump Street (2012) (id:93510), calificación: 4.752110764157091
Spotlight (2015) (id:142488), calificación: 4.7482958969479885
Night at the Opera, A (1935) (id:7132), calificación: 4.748238440478175
True Grit (1969) (id:3494), calificación: 4.676481545484244
Whale Rider (2002) (id:6385), calificación: 4.665849599974959
Mr. Smith Goes to Washington (1939) (id:954), calificación: 4.633662339335712


2. Usuario 192 - Top 10 recomendaciones:
Love and Death (1975) (id:3814), calificación: 5
21 Jump Street (2012) (id:93510), calificación: 4.757087769475153
State and Main (2000) (id:4029), calificación: 4.745794557271373
Dawn of the Dead (1978) (id:7387), calificación: 4.745351241211925
Pale Rider (1985) (id:2401), cal

C:\Users\JDEV\AppData\Local\Temp\ipykernel_5036\500994319.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  sample_top_10 = random.sample(top_10.items(), 50)
